In [1]:
import os

In [2]:
%pwd

'/home/rui/Desktop/New Folder/text-summarization/research'

In [3]:
## change directory to our root folder
os.chdir("../")

In [4]:
%pwd

'/home/rui/Desktop/New Folder/text-summarization'

In [5]:
# ! kaggle datasets download nileshmalode1/samsum-dataset-text-summarization

In [8]:
# import zipfile

# extract_to = os.path.join("samsum_dataset")

# with zipfile.ZipFile("samsum-dataset-text-summarization.zip","r") as zip_ref:
#     zip_ref.extractall(extract_to)

In [9]:
! kaggle datasets download nileshmalode1/samsum-dataset-text-summarization


Dataset URL: https://www.kaggle.com/datasets/nileshmalode1/samsum-dataset-text-summarization
License(s): unknown
samsum-dataset-text-summarization.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
from dataclasses import dataclass
from pathlib import Path

# define dataclass and inside that give parameter (frozen=True) and inside dacorator define class DataIngestion
@dataclass(frozen=True) # inside data class we can mention our varibles , and what is the type of varible
class DataIngestionConfig:
    root_dir: Path # variable : type of varibale is Path
    source_URL: str
    local_data_file:Path
    unzip_dir:Path

In [7]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yml,create_directories


In [14]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):
        

        self.config=read_yml(config_filepath)
        self.params=read_yml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) ->DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config= DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size

In [15]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exist of size: {get_size(Path(self.config.local_data_file))} ")


    def extract_zip_file(self):
        """
        zip_file_path : str
        Extract the zip file into the data directory
        Function returns None
        """

        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True) # creating director with name unzip_path
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [20]:
try: 
    config= ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-09-01 03:51:41,545: INFO : common : yaml file : config/config.yaml loaded successfully.]
[2025-09-01 03:51:41,547: INFO : common : yaml file : params.yaml loaded successfully.]
[2025-09-01 03:51:41,549: INFO : common : Created directory at : artifacts]
[2025-09-01 03:51:41,550: INFO : common : Created directory at : artifacts/data_ingestion]
[2025-09-01 03:51:41,550: INFO : 1018075320 : File already exist of size: ~ 8181 KB ]
